# Intro

LLMs are becoming increasingly integrated into development workflows,
creating demand for frameworks that make AI-based application development accessible.
In the JVM and Kotlin ecosystem, Spring is exactly such a framework.
Yes, that same Spring that has been around for over 20 years!

Spring AI is a new addition that simplifies AI integration into your Kotlin code.
Spring AI provides:

* A unified API for working with various AI models and LLM providers (OpenAI, Anthropic, Ollama, etc.)
* Components for prompt processing and context management
* Built-in capabilities for vector stores and RAG applications

In this series of posts, we'll explore the core features of Spring AI.
Following programming tradition,
our first post will be a "Hello, world!" of sorts:
we'll connect to an LLM provider and ask it to tell us a joke.

First, let's add the necessary dependencies.
> [!NOTE] We'll be working with Claude models from Anthropic,
> but we'll include commented code for OpenAI as well.
> This will help you understand the general API approach regardless of which model you choose.

In [1]:
@file:DependsOn("org.springframework.ai:spring-ai-anthropic-spring-boot-starter:1.0.0-M6")
//@file:DependsOn("org.springframework.ai:spring-ai-openai-spring-boot-starter:1.0.0-M6")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.18.2")

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY") ?: "YOUR_ANTHROPIC_API_KEY"
//val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

First, let's create a low-level API client for our chosen provider.
This only requires the API key:

In [3]:
import org.springframework.ai.anthropic.api.AnthropicApi
//import org.springframework.ai.openai.api.OpenAiApi

val anthropicApi = AnthropicApi(apiKey)
//val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()

Now that we have our client, we can make a simple model call:

In [4]:
val anthropicMessage = AnthropicApi.AnthropicMessage(
    listOf(AnthropicApi.ContentBlock("Tell me a joke")), AnthropicApi.Role.USER
)
//val openAiMessage = OpenAiApi.ChatCompletionMessage("Tell me a joke", OpenAiApi.ChatCompletionMessage.Role.USER)


anthropicApi.chatCompletionEntity(
    AnthropicApi.ChatCompletionRequest(
        AnthropicApi.ChatModel.CLAUDE_3_5_SONNET.value,
        listOf(anthropicMessage), null, 100, 0.8, false
    )
)
//openAiApi.chatCompletionEntity(
//    OpenAiApi.ChatCompletionRequest(listOf(openAiMessage), OpenAiApi.ChatModel.CHATGPT_4_O_LATEST.value, 0.8, false)
//)

<200 OK OK,ChatCompletionResponse[id=msg_015hgKST86nP1Spu1Mc9BtJu, type=message, role=ASSISTANT, content=[ContentBlock[type=TEXT, source=null, text=Here's a classic one:

Why don't scientists trust atoms?
Because they make up everything! 😄, index=null, id=null, name=null, input=null, toolUseId=null, content=null]], model=claude-3-5-sonnet-20241022, stopReason=end_turn, stopSequence=null, usage=Usage[inputTokens=11, outputTokens=27]],[:status:"200", anthropic-organization-id:"ea22ac71-22a7-43cc-9c6a-0a6a2c25768f", anthropic-ratelimit-input-tokens-limit:"1000000", anthropic-ratelimit-input-tokens-remaining:"1000000", anthropic-ratelimit-input-tokens-reset:"2025-03-24T10:55:48Z", anthropic-ratelimit-output-tokens-limit:"2000000", anthropic-ratelimit-output-tokens-remaining:"2000000", anthropic-ratelimit-output-tokens-reset:"2025-03-24T10:55:48Z", anthropic-ratelimit-requests-limit:"7000", anthropic-ratelimit-requests-remaining:"6999", anthropic-ratelimit-requests-reset:"2025-03-24T10:55:4

Congratulations, that's our first joke!

As you've noticed, this API doesn't look very convenient,
but that's the nature of low-level APIs.

Let's raise the abstraction level.
We'll create `*ChatOptions` to help us set and use parameters for our LLM provider requests:

In [5]:
import org.springframework.ai.anthropic.AnthropicChatOptions
//import org.springframework.ai.openai.OpenAiChatOptions

val anthropicOptions = AnthropicChatOptions.builder()
    .model(AnthropicApi.ChatModel.CLAUDE_3_5_SONNET)
    .temperature(0.7)
    .maxTokens(1024)
    .build()
//val openAiOptions = OpenAiChatOptions.builder()
//    .model(OpenAiApi.ChatModel.CHATGPT_4_O_LATEST)
//    .temperature(0.7)
//    .build()

Now let's create a `*ChatModel`, which will help us get another joke:

In [6]:
import org.springframework.ai.anthropic.AnthropicChatModel
//import org.springframework.ai.openai.OpenAiChatModel

val anthropicChat = AnthropicChatModel.builder()
    .anthropicApi(anthropicApi)
    .defaultOptions(anthropicOptions)
    .build()
//val openAiChat = OpenAiChatModel.builder()
//    .openAiApi(openAiApi)
//    .defaultOptions(openAiOptions)
//    .build()

Let's ask for a joke about Kotlin:

In [7]:
anthropicChat.call("Tell me a joke about Kotlin")
//openAiChat.call("Tell me a joke about Kotlin")

Here's a Kotlin joke:

Why did the Kotlin developer quit their job?

Because they couldn't handle all the null pressure! 

(A reference to Kotlin's null safety features, which help prevent the dreaded NullPointerException that Java developers often face) 😄

And now we have a joke about Kotlin!

This API is still specific to a particular provider.
You might need an even higher level of abstraction,
where your logic doesn't depend on which model you're using.

For this, we'll use the `ChatClient`:

In [8]:
import org.springframework.ai.chat.client.ChatClient

val chatClient = ChatClient.create(anthropicChat)
chatClient.prompt("Tell me a joke about Kotlin").call().content()

Here's a Kotlin joke:

Why do Kotlin developers never get lost?

Because they always have a Nullable compass! (They can check if it's null before following its direction 😉)

...I know, it's pretty bad, but hey, at least it's not as null-safe as Java jokes! 😄

Congratulations! Now we can write a simple joke and anecdote application.

Check out the next notebooks to learn more about Kotlin and Spring AI.